<a href="https://colab.research.google.com/github/repitta/CienciaDeDadosEducacionais/blob/master/Montando_a_tabela_com_os_arquivos_do_sigaa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Arquivos vindos do sigaa da ufrn para transformar em um arquivo csv com os seguintes dados : matricula, situação,ano e turma

# Pré-processamento

Importando bibliotecas

In [0]:
import pandas as pd
import numpy as np

Lendo as url's das notas

In [0]:
notas_url = ["https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/dadosUFRN/notas_discentes_2017.1.csv",
             "https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/dadosUFRN/notas_discentes_2017.2.csv",
             "https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/dadosUFRN/notas_discentes_2018.1.csv",
             "https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/dadosUFRN/notas_discentes_2018.2.csv",
             "https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/dadosUFRN/notas_discentes_2019.1.csv",
             "https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/dadosUFRN/notas_discentes_2019.2.csv"
            ]

Lendo como CSV

In [0]:
notas2017_1 = pd.read_csv(notas_url[0], sep=';', encoding = "ISO-8859-1" )
notas2017_2 = pd.read_csv(notas_url[1], sep=';', encoding = "ISO-8859-1" )
notas2018_1 = pd.read_csv(notas_url[2], sep=';', encoding = "ISO-8859-1" )
notas2018_2 = pd.read_csv(notas_url[3], sep=';', encoding = "ISO-8859-1" )
notas2019_1 = pd.read_csv(notas_url[4], sep=',', encoding = "ISO-8859-1" )
notas2019_2 = pd.read_csv(notas_url[5], sep=',', encoding = "ISO-8859-1" )

In [4]:
notas2017_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23159 entries, 0 to 23158
Data columns (total 9 columns):
periodo      23159 non-null int64
codigo       23159 non-null object
nome         23159 non-null object
turma        23159 non-null object
siape        23159 non-null object
horario      23159 non-null object
matricula    23159 non-null int64
nota         23159 non-null object
situacao     23159 non-null object
dtypes: int64(2), object(7)
memory usage: 1.6+ MB


In [5]:
notas2017_1["situacao"].unique()

array(['EXCLUIDA', 'DESISTENCIA', 'CANCELADO', 'APROVADO', 'REPROVADO',
       'APROVADO POR NOTA', 'TRANCADO',
       'REPROVADO POR MÉDIA E POR FALTAS', 'INDEFERIDO',
       'REPROVADO POR NOTA', 'REPROVADO POR FALTAS',
       'REPROVADO POR NOTA E FALTA', 'EM ESPERA'], dtype=object)

Função para converter para string

In [0]:
def converteParaString(table):
  table["codigo"]     = table["codigo"].astype(str)
  table["disciplina"] = table["disciplina"].astype(str)
  table["turma"]      = table["turma"].astype(str)
  table["horario"]    = table["horario"].astype(str)
  table["matricula"]  = table["matricula"].astype(str)
  table["situacao"]   = table["situacao"].astype(str)
  return table

In [0]:
def fazerReplace(table):
  table.loc[table["situacao"].str.contains("REPROVADO"),"situacao"] = "REPROVADO"
  table.loc[table["situacao"].str.contains("APROVADO"),"situacao"] = "APROVADO"
  return table

Função filtro para pegar apenas as situações onde o aluno conseguiu se matricular na disciplina

In [0]:
def filtroSituacoesValidas(table):
  table = table.loc[ (table["situacao"].str.contains("APROVADO")) |
                     (table["situacao"].str.contains("REPROVADO")) 
                   ] 
  #tem disciplina que só contam para a horas complementares .. não tem nota
  table = table.loc[ ~(table["nota"].str.contains("-")) ]
  table["nota"]   = table["nota"].astype(float)
  
  return table

Função para adicionar os anos a cada semestre

In [0]:
def concatenaAno(table,i,ano):
  if i%2==0:
    ano += 1
  table = fazerReplace(table)  
  table["ano"] = str(ano)+"."+str(table["periodo"].iloc[0])
  table.rename(columns={"nome":"disciplina"},inplace=True)
  table.drop(columns=["periodo","siape"], inplace=True)
  table = filtroSituacoesValidas(table)
  table = converteParaString(table)
  i+=1
  return (table,i,ano)

Chamada da função de inserir o ano + período, nela também é chamada a de filtro de situação e a de conversão para string

In [0]:
ano = 2016
i = 2
(notas2017_1,i,ano) = concatenaAno(notas2017_1,i,ano)
(notas2017_2,i,ano) = concatenaAno(notas2017_2,i,ano)
(notas2018_1,i,ano) = concatenaAno(notas2018_1,i,ano)
(notas2018_2,i,ano) = concatenaAno(notas2018_2,i,ano)
(notas2019_1,i,ano) = concatenaAno(notas2019_1,i,ano)
(notas2019_2,i,ano) = concatenaAno(notas2019_2,i,ano)

In [11]:
notas2017_1.head()

,codigo,disciplina,turma,horario,matricula,nota,situacao,ano
41,ECT2106,"CIÊNCIA, TECNOLOGIA E SOCIEDADE",1,4M34 (13/02/2017 - 01/07/2017),20170000904,7.0,APROVADO,2017.1
57,ECT2104,QUÍMICA GERAL,02A,2M1256 5M56 (13/02/2017 - 01/07/2017),20170001198,2.2,REPROVADO,2017.1
66,ECT2101,PRÉ-CÁLCULO,1,2356M34 (13/02/2017 - 24/04/2017),20170001456,2.4,REPROVADO,2017.1
69,ECT2104,QUÍMICA GERAL,02A,2M1256 5M56 (13/02/2017 - 01/07/2017),20170001456,5.1,APROVADO,2017.1
71,ECT2106,"CIÊNCIA, TECNOLOGIA E SOCIEDADE",1,4M34 (13/02/2017 - 01/07/2017),20170001456,6.7,APROVADO,2017.1


In [12]:
notas2017_1["situacao"].unique()

array(['APROVADO', 'REPROVADO'], dtype=object)

# Projeto análise de solicitação de matricula

Media de todas as disciplinas

Maior nota

Menor nota

Quantidade de disciplinas

Carga horaria total

Quantidade de aprovação e reprovação

In [0]:
notas2017_1.sort_values(by="matricula",inplace=True)

In [17]:
notas2017_1.head()

,codigo,disciplina,turma,horario,matricula,nota,situacao,ano
13329,ECT2502,DIMENSÕES FILOSÓFICAS DA TECNOLOGIA MODERNA,1,2M3456 (13/02/2017 - 01/07/2017),2009028457,6.8,APROVADO,2017.1
4810,ECT2303,LINGUAGEM DE PROGRAMAÇÃO,03C,35M34 5T12 (13/02/2017 - 01/07/2017),2009028457,0.1,REPROVADO,2017.1
19889,ECT2206,"GESTÃO E ECONOMIA DA CIÊNCIA, TECNOLOGIA E INO...",6,24T34 (13/02/2017 - 01/07/2017),2009028457,5.3,APROVADO,2017.1
11394,ECT2304,INTRODUÇÃO À FÍSICA CLÁSSICA II,4,35N12 (13/02/2017 - 01/07/2017),2009028457,0.0,REPROVADO,2017.1
18569,DCO1002,ENGENHARIA DE SOFTWARE,1,46N34,2009030796,5.5,REPROVADO,2017.1


In [0]:
df = pd.DataFrame()
final = pd.DataFrame()

df = notas2017_1.groupby("matricula")["nota"].mean()
df = df.reset_index()
df.rename(columns={"nota":"media"},inplace=True)
final = df

df = notas2017_1.groupby("matricula")["nota"].min()
df = df.reset_index()
df.rename(columns={"nota":"min"},inplace=True)
final = final.merge(df,on="matricula")

df = notas2017_1.groupby("matricula")["nota"].max()
df = df.reset_index()
df.rename(columns={"nota":"max"},inplace=True)
final = final.merge(df,on="matricula")

df = notas2017_1.groupby("matricula")["codigo"].count()
df = df.reset_index()
df.rename(columns={"codigo":"QuantDisciCursadas"},inplace=True)
final = final.merge(df,on="matricula")


df = notas2017_1.groupby(["matricula","situacao"])["codigo"].count()
df = df.reset_index()
df.rename(columns={"codigo":"quanSituacao"},inplace=True)

aprovado = df.loc[df["situacao"] == "APROVADO",:]
aprovado.rename(columns={"quanSituacao":"quantAprovado"},inplace=True)
final = final.merge(aprovado,on="matricula")

reprovado = df.loc[df["situacao"] == "REPROVADO",:]
reprovado.rename(columns={"quanSituacao":"quantReprovado"},inplace=True)
final = final.merge(reprovado,on="matricula")
final = final.drop(["situacao_x","situacao_y"],axis=1)
final

# Trecho para pegar as notas do segundo e do primeiro semestre - falta fazer

Listas com as matérias que eu quero pegar

In [0]:
semestre1 = ["ECT2101","ECT2102","ECT2103","ECT2104","ECT2105","ECT2106"]
semestre2 = ["ECT2201","ECT2202","ECT2203","ECT2204","ECT2205","ECT2206","ECT2207"]

In [0]:
def selecionaDiscDoPrimeiroSemestreEumaDoSegundo(table,semestre1,semestre2,NotasSemestre1e2):
  tabelaConcatenada = pd.DataFrame()
  semestre1.append(semestre2[0])
  for i in range(len(semestre1)):
    tabelaDaVez = table[table["codigo"] == semestre1[i]]
    tabelaConcatenada = pd.concat([tabelaConcatenada,tabelaDaVez],ignore_index=True)
  semestre1.pop(-1)  
  semestre2.pop(0)
  return (tabelaConcatenada,semestre1,semestre2)    

In [0]:
NotasSemestre1e2 = pd.DataFrame()
(NotasSemestre1e2,semestre1,semestre2) = selecionaDiscDoPrimeiroSemestreEumaDoSegundo(notas2017_1,semestre1,semestre2,NotasSemestre1e2)
(NotasSemestre1e2,semestre1,semestre2) = selecionaDiscDoPrimeiroSemestreEumaDoSegundo(notas2017_2,semestre1,semestre2,NotasSemestre1e2)
(NotasSemestre1e2,semestre1,semestre2) = selecionaDiscDoPrimeiroSemestreEumaDoSegundo(notas2018_1,semestre1,semestre2,NotasSemestre1e2)
(NotasSemestre1e2,semestre1,semestre2) = selecionaDiscDoPrimeiroSemestreEumaDoSegundo(notas2018_2,semestre1,semestre2,NotasSemestre1e2)
(NotasSemestre1e2,semestre1,semestre2) = selecionaDiscDoPrimeiroSemestreEumaDoSegundo(notas2019_1,semestre1,semestre2,NotasSemestre1e2)
(NotasSemestre1e2,semestre1,semestre2) = selecionaDiscDoPrimeiroSemestreEumaDoSegundo(notas2019_2,semestre1,semestre2,NotasSemestre1e2)

In [0]:
NotasSemestre1e2

In [0]:
semestre1.append(semestre2[0])
semestre1.pop(-1)
semestre1

['ECT2101', 'ECT2102', 'ECT2103', 'ECT2104', 'ECT2105', 'ECT2106']

In [0]:
tab = tab[tab["nota"] != '-']

In [0]:
disciplinas = merge_notas2017["disciplina"].unique()

In [0]:
def tabelaParaCadaMateriaDoSegundoSemestre(table,semestre1,semestre2):
  tam = len(semestre2)
  for i in range(tam):
    semestre1.append(semestre2[i])
    

In [0]:
table["matricula"] = 0


In [0]:
def criaColunasAnoNotaSituacaoPorDisciplina(table):
  tam = len(disciplinas)
  for i in range(tam):
    table["semana "+str(i)] = 0
    table[disciplina[i] + " ano "] = 0
    table[disciplina[i] + " nota "] = 0
    table[disciplina[i] + " situacao "] = 0
  return table

In [0]:
table = criaColunasAnoNotaSituacaoPorDisciplina(notas2017_1)
table

#Parte 2

In [0]:
#gerar os arquivos com as notas de cada disciplina
cod_dis = ["ECT2101","ECT2102", "ECT2103","ECT2104", "ECT2105","ECT2106"]

#Para os semestres 2017.1 e 2017.2
for cod in cod_dis :
  csv20171 = classes[0][classes[0]['codigo']==cod]
  csv20171.to_csv(r'20171'+cod+'.csv')

csv20172 = classes[1][classes[1]['codigo']=="ECT2201"]
csv20172.to_csv(r'20172ECT2201.csv')

In [0]:
# aqui tu vai colocar o nome de todos os aquivos 
notas_url_2017 = ["20171ECT2101.csv","20171ECT2102.csv","20171ECT2103.csv","20171ECT2104.csv","20171ECT2105.csv","20171ECT2106.csv",
                  "20172ECT2201.csv","20172ECT2202.csv","20172ECT2203.csv","20172ECT2204.csv","20172ECT2205.csv","20172ECT2206.csv","20172ECT2207.csv"]


In [0]:
# Notas since 2017.1 until 2017.2
notas_1 = pd.read_csv(notas_url_2017[0] )
notas_2 = pd.read_csv(notas_url_2017[1] )
notas_3 = pd.read_csv(notas_url_2017[2] )
notas_4 = pd.read_csv(notas_url_2017[3] )

notas2017 = [notas_1,notas_2,notas_3,notas_4]

In [0]:
# Concatenating all notas
merge_notas2017 = notas2017[0]
for i in range(1,13):
  merge_notas2017 = pd.concat([merge_notas2017, notas2017[i]], ignore_index=True)
merge_notas2017

In [0]:
disciplinas = merge_notas2017["disciplina"].unique()
colunas      = ["matricula"]
tabela2017   = pd.DataFrame(columns=colunas)

for i in disciplinas:
  notasTurma = merge_notas2017.loc[merge_notas2017["disciplina"].str.contains(i, regex=False),["matricula","disciplina","ano","nota","situacao"]]
  notasTurma = notasTurma.sort_values("matricula")
  tabela2017 = pd.merge(notasTurma,tabela2017,on=['matricula'], how="outer") 


tabela2017.columns = ['matricula', 
                 disciplinas[12],disciplinas[12]+" ano ",disciplinas[12]+" nota ",disciplinas[12]+" situacao ",
                 disciplinas[11],disciplinas[11]+" ano ",disciplinas[11]+" nota ",disciplinas[11]+" situacao ",
                 disciplinas[10],disciplinas[10]+" ano ",disciplinas[10]+" nota ",disciplinas[10]+" situacao ",
                 disciplinas[9], disciplinas[9] +" ano ",disciplinas[9] +" nota ",disciplinas[9] +" situacao ",
                 disciplinas[8], disciplinas[8] +" ano ",disciplinas[8] +" nota ",disciplinas[8] +" situacao ",     
                 disciplinas[7], disciplinas[7] +" ano ",disciplinas[7] +" nota ",disciplinas[7] +" situacao ",     
                 disciplinas[6], disciplinas[6] +" ano ",disciplinas[6] +" nota ",disciplinas[6] +" situacao ",
                 disciplinas[5], disciplinas[5] +" ano ",disciplinas[5] +" nota ",disciplinas[5] +" situacao ", 
                 disciplinas[4] ,disciplinas[4] +" ano ",disciplinas[4] +" nota ",disciplinas[4] +" situacao ",
                 disciplinas[3], disciplinas[3] +" ano ",disciplinas[3] +" nota ",disciplinas[3] +" situacao ",
                 disciplinas[2], disciplinas[2] +" ano ",disciplinas[2] +" nota ",disciplinas[2] +" situacao ",
                 disciplinas[1], disciplinas[1] +" ano ",disciplinas[1] +" nota ",disciplinas[1] +" situacao ",
                 disciplinas[0], disciplinas[0] +" ano ",disciplinas[0] +" nota ",disciplinas[0] +" situacao "]


tabela2017 = tabela2017.drop_duplicates()
tabela2017 = tabela2017.sort_values('matricula')
tabela2017.head(2)

In [0]:
tabela2017 = tabela2017.replace(np.nan, "-")

# Implementação de funções pegar as matérias do segundo semestre com base nas do primeiro

In [0]:
codigos = ["ECT2201","ECT2202","ECT2203","ECT2204","ECT2205","ECT2206","ECT2207"]

In [0]:
disciplinas = [""]

In [0]:
'''for i in range(tam):
  csv = classes[i][classes[i]["codigo"]=="ECT2203"]
  csv.to_csv(r'lop'+str(i+1)+'.csv')

In [0]:
# aqui tu vai colocar o nome de todos os aquivos que tu gerou
notas_Lop = ["lop1.csv","lop2.csv","lop3.csv","lop4.csv", "lop5.csv"]

In [0]:
# Notas since 2017.1 until 2017.2
notas_1 = pd.read_csv(notas_Lop [0] )
notas_2 = pd.read_csv(notas_Lop [1] )
notas_3 = pd.read_csv(notas_Lop [2] )
notas_4 = pd.read_csv(notas_Lop [3] )
notas_5 = pd.read_csv(notas_Lop [4] )
notas_6 = pd.read_csv(notas_Lop [5] )


# Setting priginal data into one vector due to easier manipulation, aqui tu vai adicionar todas a variaveis
notas =[notas_1, notas_2,notas_3, notas_4,notas_5 ]


In [0]:
for i in range(5):
  arq =pd.DataFrame()
  for c in notas[i]["turma"].unique():
    a =  notas[i].loc[notas[i]["turma"]== c][["matricula","situacao","ano","turma"]]
    arq = arq.append(a, sort=False, ignore_index=True)
    csv_situacao = arq
   
  csv_situacao.to_csv(r'lopSituacao'+str(i+1)+'.csv')